Prometheus
----------

![](https://prometheus.io/assets/architecture.png)

Quelle: [Prometheus](https://prometheus.io) 

- - -

[Prometheus](https://prometheus.io) ist ein Open-Source-Toolkit zur Systemüberwachung und -Alamierung, das ursprünglich bei SoundCloud entwickelt wurde.

Prometheus trat der [Cloud Native Computing Foundation](https://www.cncf.io/) im Jahr 2016 als zweites gehostetes Projekt nach Kubernetes bei.

Zusammen mit zwei weiteren Produkten ist der Monitoring Open Source Stack wie folgt:

* **Prometheus** ist ein Zeitreihendatenbank- (time series database) und Überwachungstool, mit dem Metrik Endpunkte abgefragt und die von diesen Endpunkten bereitgestellten Daten abgeholt und verarbeitet werden. Daten können mit PromQL abgefragt werden.
* **Grafana** ist ein Datenvisualisierungs- und Analysetool, mit dem Sie Dashboards und Diagramme für Ihre Metrikdaten erstellen können.
* **Alertmanager**, der normalerweise zusammen mit Prometheus bereitgestellt wird, bildet die Warnschicht des Stapels. Er verarbeitet von Prometheus generierte Warnungen und dedupliziert (eliminieren von redundanten Daten), gruppiert und leitet sie z.B. via E-Mail weiter.


***
### Installation

Prometheus ist in der [duk-demo](https://github.com/mc-b/duk-demo) Umgebung bereits installiert. Siehe Projekt [duk](https://github.com/mc-b/duk/tree/master/prometheus).

Deshalb fahren wir mit dem User Interface weiter. Die Ports der User Interfaces

* Grafana (Service: prometheus-operator-grafana) 
* Prometheus (Service: prometheus-operator-prometheus)
* Alertmanager (Service: prometheus-operator-alertmanager)

sind nur innerhalb des Cluster zugänglich. Deshalb müssen wir diese zuerst freischalten bzw. gegen Aussen öffnen:

In [ ]:
! kubectl -n monitoring get service -l app.kubernetes.io/name=grafana -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! kubectl -n monitoring get service -l app=prometheus-operator-prometheus -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! kubectl -n monitoring get service -l app=prometheus-operator-alertmanager -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -

Sind Sie freigeschaltet, können wir Sie wie andere Services via Cluster-IP und Port erreichen:

In [ ]:
! echo "Grafana UI     : " $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl -n monitoring get service -l app.kubernetes.io/name=grafana -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')/g")
! echo "Prometheus UI  : " $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl -n monitoring get service -l app=prometheus-operator-prometheus -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')/g")
! echo "Alertmanager UI: " $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl -n monitoring get service -l app=prometheus-operator-alertmanager -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')/g")

**Grafana UI** Einloggen mittels User/Password **admin/prom-operator**. Auf der linken Seite kann zwischen einer Reihe von vorbereitenden Dashboards ausgewählt werden, z.B. Nodes.

In der **Prometheus** Oberfläche kann mittels der Abfragesprache [PromQL](https://prometheus.io/docs/prometheus/latest/querying/basics/) gezielt Ressourcen ausgewählt werden, z.B. durch Query von `etcd_object_counts`.

Der **Alertmanager** dient zum Verarbeiten von Warnungen. Für ein Beispiel siehe [Notifikation Beispiel](https://prometheus.io/docs/alerting/notification_examples/)

- - -
### Custom Ressourcen

Der Prometheus [Operator](https://github.com/coreos/prometheus-operator/blob/master/README.md) stellt eine Reihe von [benutzerdefinierten Ressourcen (CRDs)](https://kubernetes.io/docs/tasks/access-kubernetes-api/extend-api-custom-resource-definitions/) für das Monitoring bereit.

* **Prometheus**, für eine Prometheus Instanz.
* **Alertmanager**, für eine Alertmanager Instanz
* **ServiceMonitor**, für die Überwachung von Kubernetes-Diensten.
* **PodMonitor**, für die Überwachung von Pods.


- - -
### Eigene Services überwachen

Damit eigene (Micro)Services überwacht werden können, müssen diese Metrics Informationen liefern.

Dazu müssen die (Micro)Services Metrik Informationen liefern.

Die am Anfang vorgestellen synchronen Microservices liefern diese.

Dazu starten wir die Microservices frisch inkl. ServiceMonitor für Prometheus.

In [ ]:
%%bash
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml

cat <<%EOF% | kubectl apply --namespace ms-kubernetes -f -
apiVersion: monitoring.coreos.com/v1
kind: ServiceMonitor
metadata:
  name: microservice
  labels:
    tier: microservice
spec:
  selector:
    matchLabels:
      tier: microservice
  endpoints:
  - port: customer
%EOF%

kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service apache --namespace ms-kubernetes -o=jsonpath='{ .spec.ports[0].nodePort }')/"   

Mittels `curl` können wir die Metriken abfragen:

In [ ]:
! curl http://customer.ms-kubernetes:8080/metrics

In [ ]:
! kubectl get all,servicemonitors --namespace ms-kubernetes

- - -
Prometheus Teil:

Zuerst brauchen wir die entsprechenden Rechte     

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-kubernetes -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: microservice
---
apiVersion: rbac.authorization.k8s.io/v1beta1
kind: ClusterRole
metadata:
  name: microservice
rules:
- apiGroups: [""]
  resources:
  - nodes
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups: [""]
  resources:
  - configmaps
  verbs: ["get"]
- nonResourceURLs: ["/metrics"]
  verbs: ["get"]
---
apiVersion: rbac.authorization.k8s.io/v1beta1
kind: ClusterRoleBinding
metadata:
  name: microservice
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: microservice
subjects:
- kind: ServiceAccount
  name: microservice
  namespace: ms-kubernetes
%EOF%


### Eigene Prometheus Instanz



In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-kubernetes -f -
apiVersion: v1
kind: Service
metadata:
  name: microservice
spec:
  type: NodePort
  ports:
  - name: web
    port: 9090
    protocol: TCP
    targetPort: web
  selector:
    prometheus: prometheus
  type: LoadBalancer
---
apiVersion: monitoring.coreos.com/v1
kind: Prometheus
metadata:
  name: prometheus
spec:
  serviceAccountName: microservice
  serviceMonitorSelector:
    matchLabels:
      tier: microservice
  resources:
    requests:
      memory: 400Mi
  enableAdminAPI: true
%EOF%
